### **Advanced Data Cleaning:**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import missingno as msn
import haversine as hs
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def time_diff(row):

    order_picked = pd.to_timedelta(row['Time_Order_picked'])
    time_order =  pd.to_timedelta(row['Time_Orderd'])

    x = order_picked - time_order
    return x

def haversine_row(row):
    loc1 = (row['Restaurant_latitude'], row['Restaurant_longitude'])
    loc2 = (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
    return hs.haversine(loc1, loc2, unit=hs.Unit.KILOMETERS)



def basic_data_cleaning(df):

    df.drop(columns=['ID'],inplace=True)
    df['Restaurant_latitude'] = abs(df['Restaurant_latitude'].replace(0,np.nan))
    df['Restaurant_longitude'] = abs(df['Restaurant_longitude'].replace(0,np.nan))
    cols = ["Delivery_location_latitude","Delivery_location_longitude"]
    df.loc[df['Restaurant_latitude'].isnull(), cols] = np.nan

    columns = ["Delivery_person_Age",  "Delivery_person_Ratings","Time_Orderd", "Weatherconditions", "Road_traffic_density",
    "multiple_deliveries", "Festival", "City"]
    for i in columns:
        df[i] = df[i].replace('NaN ',np.nan)
    df['Weatherconditions'] = df['Weatherconditions'].replace("conditions NaN",np.nan)
    df['Weatherconditions'] = df['Weatherconditions'].apply(lambda x: x.split()[-1] if isinstance(x, str) else x)
    df['Time_taken(min)'] = df['Time_taken(min)'].apply(lambda x: x[-2:]).astype("float")

    # changing datatype 
    df['Delivery_person_Age'] = df['Delivery_person_Age'].astype("float")
    df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].astype("float")
    df['Order_Date'] = pd.to_datetime(df['Order_Date'])
    df['multiple_deliveries'] = df['multiple_deliveries'].astype("float")

    # applying harversine distance
    df['Distance_res_to_loc_KM'] = df[['Restaurant_latitude', 'Restaurant_longitude','Delivery_location_latitude', 'Delivery_location_longitude']].apply(haversine_row,axis=1)
    df.insert(7,"Distance_res_loc_KM",df.pop('Distance_res_to_loc_KM'))

    # finding the time taken between restrurant & order pickup
    df["Time_res_pickup"] = (df[['Time_Order_picked','Time_Orderd']].apply(time_diff,axis=1).dt.total_seconds()/60).replace({-1425.:15., -1430.:10., -1435.:5.})
    df.insert(11,"Time_res_pickup",df.pop("Time_res_pickup"))
    df["order_time_hr"] = pd.to_datetime(df['Time_Orderd']).dt.hour
    df.insert(10,"order_time_hr",df.pop("order_time_hr"))
    df["time_of_day"] = pd.cut(
    df['order_time_hr'],
    bins=[0, 6, 12, 17, 20, 24],
    labels=['after midnight', 'morning', 'afternoon', 'evening', 'night'],
    right=False
    )
    df.insert(11,"time_of_day",df.pop("time_of_day"))

    # fixing values in columns:
    df['Road_traffic_density'] = df['Road_traffic_density'].str.strip().str.lower()
    

    return df

In [3]:
df = pd.read_csv(r"E:\DATA SCIENCE & AI\PROJECTS\Swiggy Delivery Dataset\train.csv")

In [5]:
df = basic_data_cleaning(df)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Delivery_person_ID           45593 non-null  object        
 1   Delivery_person_Age          43739 non-null  float64       
 2   Delivery_person_Ratings      43685 non-null  float64       
 3   Restaurant_latitude          41953 non-null  float64       
 4   Restaurant_longitude         41953 non-null  float64       
 5   Delivery_location_latitude   41953 non-null  float64       
 6   Delivery_location_longitude  41953 non-null  float64       
 7   Distance_res_loc_KM          41953 non-null  float64       
 8   Order_Date                   45593 non-null  datetime64[ns]
 9   Time_Orderd                  43862 non-null  object        
 10  Time_Order_picked            45593 non-null  object        
 11  Time_res_pickup              43862 non-nu

In [7]:
df.isnull().sum()

Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude            3640
Restaurant_longitude           3640
Delivery_location_latitude     3640
Delivery_location_longitude    3640
Distance_res_loc_KM            3640
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Time_res_pickup                1731
Weatherconditions               616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken(min)                   0
dtype: int64

In [47]:
df["time_of_day"] = pd.cut(
    df['order_time_hr'],
    bins=[0, 6, 12, 17, 20, 24],
    labels=['after midnight', 'morning', 'afternoon', 'evening', 'night'],
    right=False
)

In [49]:
df.insert(11,"time_of_day",df.pop("time_of_day"))

In [35]:
df['order_time_hr'].isnull().sum()

np.int64(1731)

In [72]:
df['Delivery_person_Age'].describe()

count    43739.000000
mean        29.567137
std          5.815155
min         15.000000
25%         25.000000
50%         30.000000
75%         35.000000
max         50.000000
Name: Delivery_person_Age, dtype: float64

- min of age should not be 15

In [76]:
df[df['Delivery_person_Age'] <18]

,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Distance_res_loc_KM,Order_Date,Time_Orderd,...,Time_res_pickup,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
2387,JAPRES15DEL03,15.0,1.0,26.891191,75.802083,26.981191,75.892083,13.407111,2022-03-12,NaN,...,NaN,NaN,NaN,3,Meal,motorcycle,0.0,No,Urban,15.0
2905,INDORES010DEL03,15.0,1.0,22.750040,75.902847,22.810040,75.962847,9.074697,2022-04-03,NaN,...,NaN,NaN,NaN,3,Snack,scooter,1.0,No,Metropolitian,29.0
2951,SURRES17DEL03,15.0,1.0,21.149569,72.772697,21.209569,72.832697,9.122121,2022-03-21,NaN,...,NaN,NaN,NaN,3,Buffet,bicycle,1.0,No,Metropolitian,20.0
5902,CHENRES15DEL03,15.0,1.0,13.026286,80.275235,13.056286,80.305235,4.657166,2022-03-11,NaN,...,NaN,NaN,NaN,3,Drinks,bicycle,1.0,No,Metropolitian,25.0
9156,BANGRES05DEL01,15.0,1.0,12.970324,77.645748,13.080324,77.755748,17.076811,2022-03-25,NaN,...,NaN,NaN,NaN,3,Buffet,motorcycle,2.0,No,Metropolitian,34.0
10900,MUMRES14DEL02,15.0,1.0,19.181300,72.836191,19.231300,72.886191,7.646993,2022-03-30,NaN,...,NaN,NaN,NaN,3,Buffet,motorcycle,1.0,No,Metropolitian,39.0
11125,SURRES12DEL01,15.0,1.0,21.183434,72.814492,21.193434,72.824492,1.520311,2022-03-05,NaN,...,NaN,NaN,NaN,3,Meal,scooter,1.0,No,Urban,27.0
12093,INDORES03DEL02,15.0,1.0,22.751857,75.866699,22.801857,75.916699,7.562328,2022-03-24,NaN,...,NaN,NaN,NaN,3,Buffet,bicycle,0.0,No,Metropolitian,28.0
15124,RANCHIRES02DEL01,15.0,1.0,NaN,NaN,NaN,NaN,NaN,2022-03-16,NaN,...,NaN,NaN,NaN,3,Meal,bicycle,1.0,No,Metropolitian,21.0
15927,CHENRES08DEL02,15.0,1.0,13.022394,80.242439,13.072394,80.292439,7.761850,2022-03-11,NaN,...,NaN,NaN,NaN,3,Snack,bicycle,0.0,No,Metropolitian,17.0


In [ ]:
df.drop(index=df[df['Delivery_person_Age'] <18].index,inplace=True)

In [87]:
df.drop(index=df[df['Delivery_person_Ratings']>5].index,inplace=True)

In [29]:
df.insert(10,"order_time_hr",df.pop("order_time_hr"))

In [97]:
rename_city = {"INDO":"Indore","BANG":"Bengaluru","COIMB":"Coimbatore","CHEN":"Chennai","HYD":"Hyderabad",
               "RANCHI":"Ranchi","MYS":"Mysore","DEH":"Dehradun","KOC":"Kochi","PUNE":"Pune","LUD":"Ludhiana",
               "KNP":"Kanpur","MUM":"Mumbai","KOL":"Kolkata","JAP":"Jamshedpur","SUR":"Surat","GOA":"Goa",
               "AURG":"Aurangabad","AGR":"Agra","VAD":"Vadodara","ALH":"Allahabad","BHP":"Bhopal"}

In [106]:
df['Delivery_person_ID'].str.split("RES").str.get(1).unique()


array(['13DEL02 ', '18DEL02 ', '19DEL01 ', '12DEL01 ', '09DEL03 ',
       '15DEL01 ', '15DEL02 ', '05DEL02 ', '17DEL01 ', '16DEL01 ',
       '13DEL03 ', '14DEL02 ', '15DEL03 ', '01DEL01 ', '20DEL01 ',
       '14DEL01 ', '19DEL02 ', '06DEL02 ', '13DEL01 ', '02DEL01 ',
       '16DEL03 ', '04DEL01 ', '02DEL03 ', '16DEL02 ', '07DEL01 ',
       '18DEL01 ', '08DEL01 ', '17DEL02 ', '11DEL01 ', '17DEL03 ',
       '05DEL03 ', '09DEL01 ', '20DEL03 ', '08DEL03 ', '07DEL03 ',
       '12DEL03 ', '02DEL02 ', '04DEL03 ', '07DEL02 ', '12DEL02 ',
       '11DEL03 ', '010DEL03 ', '03DEL02 ', '08DEL02 ', '05DEL01 ',
       '010DEL01 ', '19DEL03 ', '03DEL03 ', '09DEL02 ', '01DEL03 ',
       '06DEL01 ', '14DEL03 ', '010DEL02 ', '03DEL01 ', '04DEL02 ',
       '01DEL02 ', '18DEL03 ', '11DEL02 ', '06DEL03 ', '20DEL02 '],
      dtype=object)

In [ ]:
df.drop(columns=['Delivery_person_ID'],inplace=True)

,rider_city,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Distance_res_loc_KM,Order_Date,Time_Orderd,...,Time_res_pickup,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,Indore,37.0,4.9,22.745049,75.892471,22.765049,75.912471,3.025153,2022-03-19,11:30:00,...,15.0,Sunny,high,2,Snack,motorcycle,0.0,No,Urban,24.0
1,Bengaluru,34.0,4.5,12.913041,77.683237,13.043041,77.813237,20.183558,2022-03-25,19:45:00,...,5.0,Stormy,jam,2,Snack,scooter,1.0,No,Metropolitian,33.0
2,Bengaluru,23.0,4.4,12.914264,77.678400,12.924264,77.688400,1.552760,2022-03-19,08:30:00,...,15.0,Sandstorms,low,0,Drinks,motorcycle,1.0,No,Urban,26.0
3,Coimbatore,38.0,4.7,11.003669,76.976494,11.053669,77.026494,7.790412,2022-04-05,18:00:00,...,10.0,Sunny,medium,0,Buffet,motorcycle,1.0,No,Metropolitian,21.0
4,Chennai,32.0,4.6,12.972793,80.249982,13.012793,80.289982,6.210147,2022-03-26,13:30:00,...,15.0,Cloudy,high,1,Snack,scooter,1.0,No,Metropolitian,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,Jamshedpur,30.0,4.8,26.902328,75.794257,26.912328,75.804257,1.489848,2022-03-24,11:35:00,...,10.0,Windy,high,1,Meal,motorcycle,0.0,No,Metropolitian,32.0
45589,Agra,21.0,4.6,NaN,NaN,NaN,NaN,NaN,2022-02-16,19:55:00,...,15.0,Windy,jam,0,Buffet,motorcycle,1.0,No,Metropolitian,36.0
45590,Chennai,30.0,4.9,13.022394,80.242439,13.052394,80.272439,4.657202,2022-03-11,23:50:00,...,15.0,Cloudy,low,1,Drinks,scooter,0.0,No,Metropolitian,16.0
45591,Coimbatore,20.0,4.7,11.001753,76.986241,11.041753,77.026241,6.232402,2022-03-07,13:35:00,...,5.0,Cloudy,high,0,Snack,motorcycle,1.0,No,Metropolitian,26.0


In [110]:
df['month_no'] = df['Order_Date'].dt.month

In [112]:
df.insert(10,'month_no',df.pop('month_no'))

In [ ]:
df['order_day'] = df['Order_Date'].dt.day_name()


0         Saturday
1           Friday
2         Saturday
3          Tuesday
4         Saturday
           ...    
45588     Thursday
45589    Wednesday
45590       Friday
45591       Monday
45592    Wednesday
Name: Order_Date, Length: 45502, dtype: object

In [120]:
df.insert(11,'order_dayname',df['Order_Date'].dt.day_name())

In [127]:

df.insert(12,'order_in_weekend',np.where(df['order_dayname'].isin(['Saturday','Sunday']),1,0))

In [124]:
df[['order_in_weekend','order_dayname']]

,order_in_weekend,order_dayname
0,1,Saturday
1,0,Friday
2,1,Saturday
3,0,Tuesday
4,1,Saturday
...,...,...
45588,0,Thursday
45589,0,Wednesday
45590,0,Friday
45591,0,Monday


In [128]:
df.columns

Index(['Delivery_person_ID', 'rider_city', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Distance_res_loc_KM', 'Order_Date',
       'month_no', 'order_dayname', 'order_in_weekend', 'Time_Orderd',
       'order_time_hr', 'time_of_day', 'Time_Order_picked', 'Time_res_pickup',
       'Weatherconditions', 'Road_traffic_density', 'Vehicle_condition',
       'Type_of_order', 'Type_of_vehicle', 'multiple_deliveries', 'Festival',
       'City', 'Time_taken(min)'],
      dtype='object')

In [2]:
def time_diff(row):

    order_picked = pd.to_timedelta(row['Time_Order_picked'])
    time_order =  pd.to_timedelta(row['Time_Orderd'])

    x = order_picked - time_order
    return x

def haversine_row(row):
    loc1 = (row['Restaurant_latitude'], row['Restaurant_longitude'])
    loc2 = (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
    return hs.haversine(loc1, loc2, unit=hs.Unit.KILOMETERS)


def basic_data_cleaning(df):

    df.drop(columns=['ID'],inplace=True)
    df['Restaurant_latitude'] = abs(df['Restaurant_latitude'].replace(0,np.nan))
    df['Restaurant_longitude'] = abs(df['Restaurant_longitude'].replace(0,np.nan))
    cols = ["Delivery_location_latitude","Delivery_location_longitude"]
    df.loc[df['Restaurant_latitude'].isnull(), cols] = np.nan

    columns = ["Delivery_person_Age",  "Delivery_person_Ratings","Time_Orderd", "Weatherconditions", "Road_traffic_density",
    "multiple_deliveries", "Festival", "City"]
    for i in columns:
        df[i] = df[i].replace('NaN ',np.nan)
    df['Weatherconditions'] = df['Weatherconditions'].replace("conditions NaN",np.nan)
    df['Weatherconditions'] = df['Weatherconditions'].apply(lambda x: x.split()[-1] if isinstance(x, str) else x)
    df['Time_taken(min)'] = df['Time_taken(min)'].apply(lambda x: x[-2:]).astype("float")

    # changing datatype 
    df['Delivery_person_Age'] = df['Delivery_person_Age'].astype("float")
    df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].astype("float")
    df['Order_Date'] = pd.to_datetime(df['Order_Date'])
    df['multiple_deliveries'] = df['multiple_deliveries'].astype("float")

    # applying harversine distance
    df['Distance_res_to_loc_KM'] = df[['Restaurant_latitude', 'Restaurant_longitude','Delivery_location_latitude', 'Delivery_location_longitude']].apply(haversine_row,axis=1)
    df.insert(7,"Distance_res_loc_KM",df.pop('Distance_res_to_loc_KM'))

    # finding the time taken between restrurant & order pickup
    df["Time_res_pickup"] = (df[['Time_Order_picked','Time_Orderd']].apply(time_diff,axis=1).dt.total_seconds()/60).replace({-1425.:15., -1430.:10., -1435.:5.})
    df.insert(11,"Time_res_pickup",df.pop("Time_res_pickup"))
    df["order_time_hr"] = pd.to_datetime(df['Time_Orderd']).dt.hour
    df.insert(10,"order_time_hr",df.pop("order_time_hr"))
    df["time_of_day"] = pd.cut(
    df['order_time_hr'],
    bins=[0, 6, 12, 17, 20, 24],
    labels=['after midnight', 'morning', 'afternoon', 'evening', 'night'],
    right=False
    )
    df.insert(11,"time_of_day",df.pop("time_of_day"))

    # fixing values in columns:
    df['Road_traffic_density'] = df['Road_traffic_density'].str.strip().str.lower()
    df['Weatherconditions'] = df['Weatherconditions'].str.strip().str.lower()
    df['Type_of_order'] = df['Type_of_order'].str.strip().str.lower()
    df['Type_of_vehicle'] = df['Type_of_vehicle'].str.strip().str.lower()
    df['Festival'] = df['Festival'].str.strip().str.lower()
    df['City'] = df['City'].str.strip().str.lower()

    # dropping some rows
    df.drop(index=df[df['Delivery_person_Age'] <18].index,inplace=True)
    df.drop(index=df[df['Delivery_person_Ratings']>5].index,inplace=True)
    
    # creating rider city info
    rename_city = {"INDO":"Indore","BANG":"Bengaluru","COIMB":"Coimbatore","CHEN":"Chennai","HYD":"Hyderabad",
               "RANCHI":"Ranchi","MYS":"Mysore","DEH":"Dehradun","KOC":"Kochi","PUNE":"Pune","LUD":"Ludhiana",
               "KNP":"Kanpur","MUM":"Mumbai","KOL":"Kolkata","JAP":"Jamshedpur","SUR":"Surat","GOA":"Goa",
               "AURG":"Aurangabad","AGR":"Agra","VAD":"Vadodara","ALH":"Allahabad","BHP":"Bhopal"}
    
    df['rider_city'] = df['Delivery_person_ID'].str.split("RES").str.get(0).replace(rename_city)
    df.insert(1,'rider_city',df.pop('rider_city'))
    df.insert(11,'order_dayname',df['Order_Date'].dt.day_name())
    df.insert(12,'order_in_weekend',np.where(df['order_dayname'].isin(['Saturday','Sunday']),1,0))
    

    df.drop(columns=['Delivery_person_ID'],inplace=True)
    
    return df

In [3]:
df = pd.read_csv(r"E:\DATA SCIENCE & AI\PROJECTS\Swiggy Delivery Dataset\train.csv")

In [4]:
clean_df = basic_data_cleaning(df)

In [5]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45502 entries, 0 to 45592
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   rider_city                   45502 non-null  object        
 1   Delivery_person_Age          43648 non-null  float64       
 2   Delivery_person_Ratings      43594 non-null  float64       
 3   Restaurant_latitude          41872 non-null  float64       
 4   Restaurant_longitude         41872 non-null  float64       
 5   Delivery_location_latitude   41872 non-null  float64       
 6   Delivery_location_longitude  41872 non-null  float64       
 7   Distance_res_loc_KM          41872 non-null  float64       
 8   Order_Date                   45502 non-null  datetime64[ns]
 9   Time_Orderd                  43862 non-null  object        
 10  order_dayname                45502 non-null  object        
 11  order_in_weekend             45502 non-null  i

___________________________________________